In [4]:
import test_subcomp_a as testing
import datetime
import cftime
import xarray as xr
import numpy as np

In [6]:
import subcomp_a_process_climate_model_data

In [2]:
data_path = analysis_parameters.DIR_INTERMEDIATE_PROCESSED_MODEL_DATA
scenario='ssp126'
normalized = False
WORKING_DIR

In [18]:
ds

NameError: name 'ds' is not defined

In [17]:
test_get_scenario_fnames(data_path, scenario, normalized)

In [11]:
names

True

In [6]:
names[0]

'tas_ssp126_MRI-ESM2-0'

In [ ]:
"""
This module processes raw CMIP6 output to create files with consistent dimensions for all models.

Running this script for all models with just averaging over ensembles, regridding,
and saving takes about 15-20 mins.

Make sure to clear the directory you would like the zarr files to be saved in - something about
this format makes it not work to overwrite existing files.
"""

####################### Load Packages ########################################

import cftime
import xarray as xr
import xesmf as xe
import numpy as np
import analysis_parameters

############ Read in Settings for Data Dictionary#############################

THIS_EXPERIMENT_ID = analysis_parameters.EXPERIMENT_LIST
THIS_VARIABLE_ID = analysis_parameters.VARIABLE_ID
OUTPUT_PATH = analysis_parameters.DIR_PROCESSED_DATA
DIR_INTERMEDIATE = analysis_parameters.DIR_INTERMEDIATE_PROCESSED_MODEL_DATA

########### Create functions for analysis ####################################

def create_reference_grid(reference_key, dset_dict):
    """Creates reference grid to which all model output will be regridded"""
    thisdata = dset_dict[reference_key]
    ds_out = xr.Dataset({'lat': thisdata['lat'],
                         'lon': thisdata['lon']})
    return ds_out

def reindex_time(startingtimes):
    """Reindexes time series to proleptic Gregorian calendar type"""
    newtimes = np.empty(np.shape(startingtimes.values),dtype=cftime.DatetimeProlepticGregorian)
    for i in range(0, len(startingtimes)):
        yr = int(str(startingtimes.values[i])[0:4])
        mon = int(str(startingtimes.values[i])[5:7])
        newdate = cftime.DatetimeProlepticGregorian(yr, mon, 15)
        newtimes[i] = newdate
    return newtimes

def regrid_model(ds, reference_grid, latvariable='lat',
                 lonvariable='lon', regrid_method='nearest_s2d'):
    """Regrids model output to a reference grid"""
    data_series = ds[THIS_VARIABLE_ID]
    ds_in = xr.Dataset({'lat': data_series[latvariable],
                        'lon': data_series[lonvariable],
                        'time': data_series['time'],
                        THIS_VARIABLE_ID: data_series})
    regridder = xe.Regridder(ds_in, reference_grid, regrid_method, periodic=True)
    data_series_regridded = regridder(ds_in)
    data_series_regridded.attrs.update(data_series.attrs)
    return data_series_regridded

def process_dataset(this_key, dset_dict, final_grid):
    """ Processes each dataset in dictionary
    This processing involves:
        (1) Averaging over all ensemble members (member ids)
        (2) Getting into consistent time format
        (3) Renaming coordinates if necessary
        (4) Regridding to reference dataset
    """
    # Get original dataset from dictionary
    ds_original = dset_dict[this_key]

    # Average over all ensemble members
    ds = ds_original.mean(dim=['member_id'])

    # Reindex time to consistent time datatype
    ds = xr.decode_cf(ds)
    newtimes = reindex_time(startingtimes=ds['time'])
    ds['time'] = xr.DataArray(newtimes, coords=[newtimes], dims=['time'])
    ds = ds.copy()

    # Rename latitude and longitude coordinate names if necessary
    if 'latitude' in ds.dims:
        ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
    else:
        pass

    # Regrid dataset to reference grid
    dataset_regridded = regrid_model(ds, final_grid)

    return dataset_regridded

def generate_new_filename(this_key):
    """Generates filename for processed data"""
    [_, _, source_id, experiment_id, _, _] = this_key.split('.')
    this_fname = THIS_VARIABLE_ID+'_'+experiment_id+'_'+source_id
    return this_fname

##################### Main Workflow ##########################################

def process_all_files_in_dictionary

In [3]:
"""
Tests for subcomponent a
"""

# Import modules
import datetime
import cftime
import os
import glob
import xarray as xr
import numpy as np

#import analysis_parameters
#import subcomp_a_create_data_dict
import subcomp_a_process_climate_model_data as process_data

# Define directory and file names
TEST_KEY1 = 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn'
TEST_KEY2 = 'CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn'

TESTING_DATA_DIR = '/home/jovyan/local-climate-data-tool/Data/files_for_testing/raw_data/'
TESTING_OUTPUT_DIR = '/home/jovyan/local-climate-data-tool/Data/files_for_testing/processed_model_data/'
VARNAME = 'tas'
TEST_INDS = [44, 120, 0]
EXP_TYPES = np.array([xr.core.dataarray.DataArray,
                      np.ndarray,
                      np.float64,
                      cftime._cftime.DatetimeProlepticGregorian])

# Read testing data into dictionary
DSET_DICT = dict()
DSET_DICT[TEST_KEY1] = xr.open_dataset(TESTING_DATA_DIR+TEST_KEY1+'.nc')
DSET_DICT[TEST_KEY2] = xr.open_dataset(TESTING_DATA_DIR+TEST_KEY2+'.nc')

In [4]:
import xarray as xr
ds_original = DSET_DICT[TEST_KEY1]
ds_ref_grid = process_data.create_reference_grid(dset_dict=DSET_DICT,
                                                 reference_key=TEST_KEY2)
ds_processed = process_data.process_dataset(this_key=TEST_KEY1,
                                            dset_dict=DSET_DICT,
                                            final_grid=ds_ref_grid)
ds_processed.load()
ds_processed=ds_processed.copy(deep=True)
    # Export intermediate processed dataset as zarr file
#ds_processed.chunk({'lon':10, 'lat':10, 'time':-1})
ds_processed.to_zarr(TESTING_OUTPUT_DIR+'test4'+'.zarr')
ds_read_in = xr.open_zarr(TESTING_OUTPUT_DIR+'test4'+'.zarr')

Overwrite existing file: nearest_s2d_144x192_160x320_peri.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable tas as the horizontal dimensions for this dataset.


ValueError: path '' contains a group

In [6]:
ds_processed.to_netcdf(TESTING_OUTPUT_DIR+'test4'+'.nc')
ds_read_in = xr.open_dataset(TESTING_OUTPUT_DIR+'test4'+'.nc')

In [9]:
print(type([0]).year))
print(type(ds_read_in['time'].values[0]))

NameError: name 'pd' is not defined

In [13]:
import pandas as pd


Int64Index([2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
            ...
            2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100],
           dtype='int64', length=1032)

True

In [ ]:


#---------------------------------------------------------------------------------
#---------------------------------------------------------------------------------
def test_reindex_time():
    """Tests whether the reindex_time function works as expected"""
    [yr, month, day] = [1850, 1, 15]
    expected_times =xr.DataArray(np.array([cftime.DatetimeProlepticGregorian(yr, month, day, 0, 0),
                                           cftime.DatetimeProlepticGregorian(yr, month+1, day, 0, 0)]))
    testdates1 = xr.DataArray(np.array([np.datetime64(datetime.datetime(yr, month, day, 0, 0)),
                                        np.datetime64(datetime.datetime(yr, month+1, day, 0, 0))]))
    testdates2 = xr.DataArray(np.array([cftime.DatetimeNoLeap(yr, month, day, 0, 0),
                                        cftime.DatetimeNoLeap(yr, month+1, day, 0, 0)]))
    testdates3 = xr.DataArray(np.array([cftime.Datetime360Day(yr, month, day, 0, 0),
                                        cftime.Datetime360Day(yr, month+1, day, 0, 0)]))
    dates_to_test = [testdates1, testdates2, testdates3]

    # Run test
    dates_converted = True
    for date_to_test in dates_to_test:
        newtimes = process_data.reindex_time(date_to_test)
        if not np.all((newtimes==expected_times).values):
            dates_converted = False
        else:
            continue
    assert dates_converted

def test_generate_new_filename(test_key=TEST_KEY1):
    """Test that the generate_new_filename function generates a string"""
    fname = process_data.generate_new_filename(test_key)
    assert ((fname is not None) and (type(fname) == str))

def test_create_reference_grid(dset_dict=DSET_DICT, test_key=TEST_KEY2):
    """ Tests that the reference grid function creates a sensible reference grid"""
    ds_original = dset_dict[test_key]
    ds_ref_grid = process_data.create_reference_grid(dset_dict=dset_dict,
                                                     reference_key=test_key)

    # Check that latitude and longitude dimensions haven't changed in regridding
    correct_dim_size = ((ds_original.dims['lat'] == ds_ref_grid.dims['lat']) and
                        (ds_original.dims['lon'] == ds_ref_grid.dims['lon']))

    # Check that reference grid doesn't have any variables other than lat and lon
    no_extra_vars = (len(ds_ref_grid.variables) == 2)

    # Check that latitude and longitude coordinates do not contain nans
    no_nans = ((not np.isnan(ds_ref_grid['lat'].values).any()) and
               (not np.isnan(ds_ref_grid['lon'].values).any()))

    #Check that latitude and longitude arrays are 1D
    one_dim = ((np.size(ds_ref_grid['lat'].values) == np.shape(ds_ref_grid['lat'].values)[0]) and
               (np.size(ds_ref_grid['lon'].values) == np.shape(ds_ref_grid['lon'].values)[0]))

    # Check that latitude and longitude arrays are big enough
    min_coord_size = 20
    big_enough = ((np.size(ds_ref_grid['lat'].values) > min_coord_size) and
                  (np.size(ds_ref_grid['lon'].values) > min_coord_size))

    assert correct_dim_size
    assert no_extra_vars
    assert no_nans
    assert one_dim
    assert big_enough

def test_regrid_model_dims(dset_dict=DSET_DICT, key_to_regrid=TEST_KEY1,
                           key_for_grid=TEST_KEY2):
    """Tests that the regrid function results in an array of the right dimensions"""
    
    # Regrid model output
    ds_to_regrid = dset_dict[key_to_regrid]
    reference_grid = process_data.create_reference_grid(dset_dict=dset_dict,
                                                        reference_key=key_for_grid)
    ds_regridded = process_data.regrid_model(ds_to_regrid,
                                             reference_grid)
    
    # Check that lat/lonregridded data actually aligns with reference grid
    correct_lat_dim = np.size(ds_regridded['lat'].values) == np.size(reference_grid['lat'].values)
    correct_lon_dim = np.size(ds_regridded['lon'].values) == np.size(reference_grid['lon'].values)
    # Check that time dimensions have not changed
    correct_time_dim = np.equal(ds_regridded['time'].values,ds_to_regrid['time'].values).all()
    assert (correct_lat_dim and correct_lon_dim and correct_time_dim)

def test_regrid_model_nans(dset_dict=DSET_DICT, key_to_regrid=TEST_KEY1,
                           key_for_grid=TEST_KEY2, varname=VARNAME):
    """Tests that the regrid function doesn't create any nans"""
    ds_to_regrid = dset_dict[key_to_regrid]
    reference_grid = process_data.create_reference_grid(dset_dict=dset_dict,
                                                        reference_key=key_for_grid)
    ds_regridded = process_data.regrid_model(ds_to_regrid,
                                             reference_grid)

    # Check that there aren't nans if there weren't in original array
    if np.isnan(ds_to_regrid[varname].values).any():
        raise ValueError('NANs were in original array')
    else: 
        assert not np.isnan(ds_regridded[varname].values).any()

def test_process_dataset(dset_dict=DSET_DICT,
                         key_to_process=TEST_KEY1,
                         key_for_grid=TEST_KEY2,
                         test_inds=TEST_INDS,
                         expected_types=EXP_TYPES):
    """Tests whether the processed dataset output by the process_dataset
    function has expected coordinates and dimensions and plausible values"""
    #------------------ Process test dataset------------------------------------
    ds_original = dset_dict[key_to_process]
    ds_ref_grid = process_data.create_reference_grid(dset_dict=dset_dict,
                                                     reference_key=key_for_grid)
    ds_processed = process_data.process_dataset(this_key=key_to_process,
                                                dset_dict=dset_dict,
                                                final_grid=ds_ref_grid)

    #------------------ Run tests on processed dataset-------------------------
    # Check that lat/lon/time coordinate names are right
    coord_names_pass = check_coord_names(ds_processed, ds_coords_expected=['time', 'lon', 'lat'])

    # Check that time is reasonable (e.g. )
    years_pass = check_years(ds_processed, min_year=1849, max_year=2200)

    # Check that lat, lon, and time dimensions are what we expect
    dims_pass = check_dims(ds_processed, ds_original, ds_ref_grid)

    # Check that coordinate types are right
    coord_types_pass = check_coord_types(ds_processed, expected_types)

    assert coord_names_pass
    assert years_pass
    assert dims_pass
    assert coord_types_pass

def test_save_dataset(dset_dict=DSET_DICT,
                      data_path_out=TESTING_OUTPUT_DIR,
                      key_for_grid=TEST_KEY2,
                      exceptions_list=(),
                     expected_types=EXP_TYPES):
    final_grid = process_data.create_reference_grid(dset_dict=dset_dict,
                                                    reference_key=key_for_grid)
    
    delete_zarr_files(data_dir=data_path_out, regex='*')
    
    process_data.process_all_files_in_dictionary(dset_dict, exceptions_list,
                                        final_grid, data_path_out)
    endcut = -1*len('.zarr')
    begcut = len(data_path_out)
    names = [f[begcut:endcut] for f in glob.glob(data_path_out +'*.zarr')]
    files_are_saved = bool(len(names) > 0)
    
    coord_names_pass = True
    years_pass = True
    coord_types_pass = True
    for fname in names:
        filename = data_path_out + fname + '.zarr'
        ds = xr.open_zarr(filename)
        ds = xr.decode_cf(ds)
        
        # Check that ds looks the same as the processed dataset
        coord_names_pass = coord_names_pass and check_coord_names(ds, ds_coords_expected=['time', 'lon', 'lat'])
        coord_types_pass = coord_types_pass and check_coord_types(ds, expected_types)
        years_pass = years_pass and check_years(ds, min_year=1849, max_year=2200)
    
    assert files_are_saved
    assert coord_names_pass
    assert years_pass
    assert coord_types_pass

def check_coord_names(ds_processed, ds_coords_expected):
    """Checks whether coordinate names of ds are expected names"""
    coords_list = []
    for coord in ds_processed.coords:
        coords_list.append(coord)
    return bool(set(coords_list) == set(ds_coords_expected))

def check_dims(ds_processed, ds_original, ds_ref_grid):
    """Checks whether dimensions of dataset are expected based on
    regridding and original dataset dimensions"""
    if ds_processed.dims['time'] == ds_original.dims['time']:
        if  ds_processed.dims['lat'] == ds_ref_grid.dims['lat']:
            if ds_processed.dims['lon'] == ds_ref_grid.dims['lon']:
                return True
            else:
                print('Incorrect lon dimension')
                return False
        else:
            print('Incorrect lat dimension')
            return False
    else:
        print('Incorrect time dimension')
        return False

def check_years(ds_processed, min_year, max_year):
    """ Check that times are within range of plausible years for
    model output"""
    if ds_processed['time'].values[0].year > min_year:
        if ds_processed['time'].values[0].year < max_year:
            return True
        else:
            print('Start year is too big')
            return False
    else:
        print('Start year is too small')
        return False

def check_coord_types(ds_processed, expected_types):
    """Checks that processed dataset consists of coordinates
    of expected data types"""
    [exp_type_dim, exp_type_dim_value, exp_type_latlon, exp_type_time] = expected_types

    time_types_pass = (isinstance(ds_processed['time'].values[0], exp_type_time) and
                       isinstance(ds_processed['time'], exp_type_dim) and
                       isinstance(ds_processed['time'].values, exp_type_dim_value))

    lat_types_pass = (isinstance(ds_processed['lat'].values[0], exp_type_latlon) and
                      isinstance(ds_processed['lat'], exp_type_dim) and
                      isinstance(ds_processed['lat'].values, exp_type_dim_value))

    lon_types_pass = (isinstance(ds_processed['lon'].values[0], exp_type_latlon) and
                      isinstance(ds_processed['lon'], exp_type_dim) and
                      isinstance(ds_processed['lon'].values, exp_type_dim_value))

    return bool(time_types_pass and lat_types_pass and lon_types_pass)

def delete_zarr_files(data_dir, regex):
    """Deletes zarr files matching regular expression. This is a
    necessary function because zarr files cannot be overwritten"""
    i = 0
    for file in glob.glob(data_dir+regex+'.zarr'):
        os.system('rm -rf '+file)
        i = i+1
    print('deleted '+str(i)+' files in '+data_dir)
